# Basic MTPtg/Rotator integration test

This notebook performs a simple integration test between the Maint Telescope pointing component (MTPtg) and the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew. 

At the end, there is a section showing how to recover the Rotator CSC from `FAULT` state. This last part can be ignored as needed.

**NOTE:** 

Use `TestMTCS.ipynb` to see how to perform a similar set of operations but using the MTCS high level class. 

In [ ]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

In [ ]:
test_message = "Basic MTPtg/Rotator integration test"

In [ ]:
d = salobj.Domain()

In [ ]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")

In [ ]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task)

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
# Here is a small trick to get the ra before transit. Get `timeAndDate` from the pointing component, then use `RA = lst - delta`.

time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=5.)

lst = Angle(time_data.lst, unit=u.hour)
delta = Angle(10., unit=u.arcminute)
ra = lst + delta
print("=======================")
print(f"LST: {lst}")
print(f"delta: {delta}")
print(f"RA: {ra}")
print("=======================")

dec = location.lat - Angle(6., unit=u.deg)
print(f"Dec: {dec}")

target_name="Rotator test"
radec = ICRS(ra, dec)

curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print(180.-alt_az.alt.deg)

script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"{test_message}:[START]")

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 10s")

await asyncio.sleep(10.)

print("Test done")

await rot.cmd_stop.start(timeout=30)

await asyncio.sleep(0.1)

await mtptg.cmd_stopTracking.start(timeout=30)

script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"{test_message}:[END]")


# Fault State Recovery

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await asyncio.sleep(5)
await salobj.set_summary_state(mtptg, salobj.State.STANDBY)
await salobj.set_summary_state(rot, salobj.State.STANDBY)